In [9]:
###### INCLUSION CRITERIA ANALYSIS - INFORMATION GAIN FEATURES SELECTION###
####The below code 
#Creators: Xiaoru Dong
#Contributors: Linh Hoang
#Last updated: 04/8/2019
#The below Python script is used to generate features and to create the Weka input files 
#corresponding to the information gain feature selection approach that we implemented in the study

In [10]:
from collections import OrderedDict
import pandas as pd
import os

######GLOBAL FUNCTIONS TO EXTRACT FEATURE AND TO GENERATE WEKA INPUT FILES####

#Set a function for outputing features
def writefeatures(path, features):
    with open(path,"w") as fp: 
        for word in features:
            fp.write(word + "\n")
            
#Set a function to get an arff file used as the input in Weka
def writewekainput(path, features, lineNum, eachline, rct):
    with open(path,"w") as fp:        #this line of code to create a new file
        fp.write('''@RELATION classification_analysis\n''')
    
        for i in range(len(features)): 
            fp.write('''@ATTRIBUTE ''' + "word" + str(i) + ''' NUMERIC\n''') 
    
        fp.write('''@ATTRIBUTE class {1.0, 0.0}\n''')
        fp.write('''@DATA\n''')

        #check if a word appear in the feature list or not. If yes, then write "1", if no then write "0"
        for i in range(lineNum): 
            binary = {}
            for word in features:
                if word in eachline[i]:
                    binary[word] = 1
                else:
                    binary[word] = 0
                fp.write(str(binary[word]) + ",")
            fp.write(rct[i] + "\n")

#Get the path of the folder that contains the annotation file
#Change this path to the actual path of your local machine
path = "/Users/phuong/Downloads/Inclusion_Criteria_Revised/"

#Read the inclusion criteria annotation file
file = path + 'Inclusion_Criteria_Annotation_New.csv'
df = pd.read_csv(file, encoding = "ISO-8859-1")
lineNum = len(df)

#The list of stop words
stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", 
              "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 
              'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 
              'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 
              'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 
              'the', 'if', 'because', 'as', 'until', 'of', 'at', 'by', 'for', 'with', 'about', 'between', 'into',
              'through', 'during', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 
              'under', 'again', 'further', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'few', 'more', 
              'most', 'some', 'such', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 
              'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain']

In [11]:
# remove same meaning words
allwords_noredundant_features = []
wordlist_noredundant_oneline = []
rct = []
wordlist_noredundant_alllines = []

#replace the same meaning words
quasi = ['quasi-rcts', 'quasi-randomized', 'quasi-randomised', 'quasi-random', 'quasi-randomly']
cct = ['cct', 'ccts']
cba = ['cba', 'cbas', 'before-after']
privative = ["not","didn't", "don't", "won't", "isn't", "aren't", "wasn't", "weren't", "needn't", "wouldn't", "shouldn't", "can't", "couldn't", "no"]

# split the sentence into words and save in a list
for i in range(lineNum): 
    
    string_rct = str(df['Only RCTs'][i])
    if string_rct == 'nan':
        string_rct = '0.0'
    if string_rct == 'x':
        string_rct = '1.0'
    rct.append(string_rct)
    
    string = str(df['Inclusion Criteria'][i]).lower().strip()
    wordlist_noredundant_oneline = string.split(' ')
    
    for j in range(len(wordlist_noredundant_oneline)):
        wordlist_noredundant_oneline[j] = wordlist_noredundant_oneline[j].replace('.', '')
        wordlist_noredundant_oneline[j] = wordlist_noredundant_oneline[j].replace(')', '')
        wordlist_noredundant_oneline[j] = wordlist_noredundant_oneline[j].replace('(', '')
        wordlist_noredundant_oneline[j] = wordlist_noredundant_oneline[j].replace(',', '')
        wordlist_noredundant_oneline[j] = wordlist_noredundant_oneline[j].replace("'", '')
        wordlist_noredundant_oneline[j] = wordlist_noredundant_oneline[j].replace('"', '')
        if wordlist_noredundant_oneline[j] in quasi:
            wordlist_noredundant_oneline[j] = 'quasi-rcts'
        if wordlist_noredundant_oneline[j] in cct:
            wordlist_noredundant_oneline[j] = 'cct'
        if wordlist_noredundant_oneline[j] in cba:
            wordlist_noredundant_oneline[j] = 'cba'
        if wordlist_noredundant_oneline[j] in privative:
            wordlist_noredundant_oneline[j] = 'not'
        allwords_noredundant_features.append(wordlist_noredundant_oneline[j])
    wordlist_noredundant_alllines.append(wordlist_noredundant_oneline)
    
allwords_noredundant_features = list(OrderedDict.fromkeys(allwords_noredundant_features))
allwords_noredundant_features = [x for x in allwords_noredundant_features if x] #remove empty element in the list
allwords_noredundant_features = [word for word in allwords_noredundant_features if word not in stop_words]

In [12]:
f = open(path + "InformativeWords","r")
lines = f.readlines()
f.close()

lines = lines[21:]
newlines=[]
for string in lines:
    if string != "\n":
        newlines.append(string)


info_codelist = []
for i in range(len(newlines) - 1):
    string = newlines[i]
    if float(string.split(' ')[1]) > 0:
        info_codelist.append(int(string.split(' ')[-2])-1)
        

# read the informative words and save them in a list
informationgain_features = []
for word in (info_codelist):
    informationgain_features.append(allwords_noredundant_features[word])
    
# output features
informationgain_features_savepath = path + "WordsSelectedByInformationGain.csv"
writefeatures(informationgain_features_savepath, informationgain_features)

# output the weka input file
wekainput_informationgain_path= path + "WordsSelectedByInformationGain_weka_input.arff"
writewekainput(wekainput_informationgain_path, informationgain_features, lineNum, wordlist_noredundant_alllines, rct)

# We use the weka to read the weka input file. Then, we could use weka to get the results 
# about precision, recall and F-meaure base in 3 algorithms: Random Forest, J48 and Naïve Bayes.